In [1]:
import pandas as pd
from scipy.io import arff
import numpy as np
from scipy import stats
import tensorflow as tf

from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dense, Input, Dropout
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score, accuracy_score, matthews_corrcoef
import erlc
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
import time
from tensorflow.keras.utils import to_categorical


## Data
In this section we process and clean the data.

In [2]:
print('Loading dataset #1')
data = pd.DataFrame(arff.loadarff('input/power_multiclass/data1.arff')[0])

for i in range(2,16):
    print('Loading dataset #{}'.format(i))
    dataTemp = pd.DataFrame(arff.loadarff('input/power_multiclass/data{}.arff'.format(i))[0])
    data = pd.concat([data,dataTemp],axis=0)

print("Finished Loading. Final Size = {}".format(data.shape))

Loading dataset #1
Loading dataset #2
Loading dataset #3
Loading dataset #4
Loading dataset #5
Loading dataset #6
Loading dataset #7
Loading dataset #8
Loading dataset #9
Loading dataset #10
Loading dataset #11
Loading dataset #12
Loading dataset #13
Loading dataset #14
Loading dataset #15
Finished Loading. Final Size = (78377, 129)


In [3]:
def dataProc(df):
    '''
    This function takes a dataframe and splits it into data and labels, proccesses them into numpy arrays and
    splits them into training, validation, and testing data and labels.
    '''
    df = df.astype(np.float64)
    label = df['marker'].astype(int)
    df = df.drop(['marker'], axis=1)
    
    df = df.drop(['snort_log1','snort_log2','snort_log3','snort_log4',
                'control_panel_log1','control_panel_log2','control_panel_log3','control_panel_log4',
                'relay1_log','relay2_log','relay3_log','relay4_log'], axis=1)
    
    df = df.reset_index()
    
    df = df.replace(-np.inf, 0)
    df = df.replace(np.inf, 0)

    
    # Converting to arrays
    X = np.asarray(df)
    y =  np.asarray(label)
    
    # Scaling data
    scalar = preprocessing.MinMaxScaler()
    X = scalar.fit_transform(X)
    
    return X,y

In [4]:
X,y = dataProc(data)
print('Size of X: {}'.format(X.shape))
print('Size of y: {}'.format(y.shape))

Size of X: (78377, 117)
Size of y: (78377,)


In [5]:
# model = erlc.ERLC()
# model.fit(X_train, y_train)

In [6]:
# y_pred = model.predict(X_test)
# print(y_pred.shape)

In [7]:
# print(model.__class__.__name__)
# acc = accuracy_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred, average='weighted')
# mcc = matthews_corrcoef(y_test, y_pred)
# print("Accuracy = {}, F1-score = {}, MCC = {}".format(acc, f1, mcc))

## Cross Validation
For more accurate results, we want to run cross validation on our model, as well as the other models that it's being compared to.

In [8]:
def runCV(model, X, y, numFolds=10):
    cv = KFold(numFolds, True, 1)
    acc = []
    f1 = []
    mcc = []
    trainT = []
    testT = []
    
    name = model.__class__.__name__
    print("Performing CV on {}".format(name))
    i = 1;
    
    for train, test in cv.split(X):
        X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]
        print("Train shape {}".format(X_train.shape))
        print("Test shape {}".format(X_test.shape))

        time1 = time.time()
        model.fit(X_train, y_train)
        time2 = time.time()
        trainTime= time2-time1
        
        time1 = time.time()
        y_pred = model.predict(X_test)
        time2 = time.time()
        predictTime = time2-time1
        
        acc_temp = accuracy_score(y_test, y_pred)
        f1_temp = f1_score(y_test, y_pred, average='weighted')
        mcc_temp = matthews_corrcoef(y_test, y_pred)
        
        print("Fold #{}".format(i))
        print("Accuracy = {}, F1-score = {}, MCC = {}, Train Time = {}".format(acc_temp, f1_temp, mcc_temp, trainTime))
        
        
        acc.append(acc_temp)
        f1.append(f1_temp)
        mcc.append(mcc_temp)
        trainT.append(trainTime)
        testT.append(predictTime)
        i=i+1
    
    return acc, f1, mcc, trainT, testT

In [9]:
modelDT = DecisionTreeClassifier()
modelRF = RandomForestClassifier()
modelKNN = KNeighborsClassifier()
modelERLC = erlc.ERLC()

In [12]:
MODELS = [
          modelERLC,
    modelDT, modelRF, modelKNN, 
]
NUM_FOLDS = 10
writer = pd.ExcelWriter('output/Results_{}CV2.xlsx'.format(NUM_FOLDS))

for model in MODELS:
    name = model.__class__.__name__
    acc, f1 , mcc, trainT, testT = runCV(model,X,y, numFolds = NUM_FOLDS)
    
    df = pd.DataFrame({
        'Accuracy': acc,
        'F1-Score': f1,
        'MCC': mcc,
        'Training Time': trainT,
        'Testing Time': testT
    })
    
    
    df.to_excel(writer, name)
    

writer.save()

Performing CV on ERLC
Train shape (70539, 117)
Test shape (7838, 117)
Building ERLC model
Building autoencoder
Epoch 1/500
221/221 [==============================] - 8s 37ms/step - loss: 0.0285 - mse: 0.0285 - val_loss: 0.0148 - val_mse: 0.0148
Epoch 2/500
221/221 [==============================] - 5s 24ms/step - loss: 0.0123 - mse: 0.0123 - val_loss: 0.0114 - val_mse: 0.0114
Epoch 3/500
221/221 [==============================] - 5s 21ms/step - loss: 0.0115 - mse: 0.0115 - val_loss: 0.0115 - val_mse: 0.0115
Epoch 4/500
221/221 [==============================] - 4s 19ms/step - loss: 0.0114 - mse: 0.0114 - val_loss: 0.0113 - val_mse: 0.0113
Epoch 5/500
221/221 [==============================] - 7s 31ms/step - loss: 0.0114 - mse: 0.0114 - val_loss: 0.0113 - val_mse: 0.0113
Epoch 6/500
221/221 [==============================] - 5s 25ms/step - loss: 0.0113 - mse: 0.0113 - val_loss: 0.0113 - val_mse: 0.0113
Epoch 7/500
221/221 [==============================] - 5s 22ms/step - loss: 0.0113 - 

Epoch 61/500
221/221 [==============================] - 4s 17ms/step - loss: 0.0058 - mse: 0.0058 - val_loss: 0.0055 - val_mse: 0.0055
Epoch 62/500
221/221 [==============================] - 6s 29ms/step - loss: 0.0058 - mse: 0.0058 - val_loss: 0.0056 - val_mse: 0.0056
Epoch 63/500
221/221 [==============================] - 5s 22ms/step - loss: 0.0058 - mse: 0.0058 - val_loss: 0.0056 - val_mse: 0.0056
Epoch 64/500
221/221 [==============================] - 4s 18ms/step - loss: 0.0058 - mse: 0.0058 - val_loss: 0.0055 - val_mse: 0.0055
Epoch 65/500
221/221 [==============================] - 4s 17ms/step - loss: 0.0058 - mse: 0.0058 - val_loss: 0.0055 - val_mse: 0.0055
Epoch 66/500
221/221 [==============================] - 4s 16ms/step - loss: 0.0058 - mse: 0.0058 - val_loss: 0.0055 - val_mse: 0.0055
Epoch 67/500
221/221 [==============================] - 6s 27ms/step - loss: 0.0058 - mse: 0.0058 - val_loss: 0.0055 - val_mse: 0.0055
Epoch 68/500
221/221 [==============================] -

Epoch 118/500
221/221 [==============================] - 4s 17ms/step - loss: 5.8487e-04 - mse: 5.8487e-04 - val_loss: 5.0208e-04 - val_mse: 5.0208e-04
Epoch 119/500
221/221 [==============================] - 4s 17ms/step - loss: 5.6384e-04 - mse: 5.6384e-04 - val_loss: 4.9880e-04 - val_mse: 4.9880e-04
Getting new representation of the data
Training DT on original representation
Training DT on new representation
Training RF on original representation
Training RF on new representation
Training inner DNN
Epoch 1/500
221/221 [==============================] - 5s 24ms/step - loss: 2.6489 - acc: 0.2076 - f1_m: 0.0340 - val_loss: 2.0823 - val_acc: 0.2792 - val_f1_m: 0.0691
Epoch 2/500
221/221 [==============================] - 3s 12ms/step - loss: 1.8222 - acc: 0.3418 - f1_m: 0.1314 - val_loss: 1.9152 - val_acc: 0.2672 - val_f1_m: 0.0973
Epoch 3/500
221/221 [==============================] - 3s 12ms/step - loss: 1.6176 - acc: 0.3865 - f1_m: 0.1960 - val_loss: 1.6590 - val_acc: 0.3561 - val_f

221/221 [==============================] - 5s 22ms/step - loss: 1.0119 - acc: 0.6064 - f1_m: 0.5664 - val_loss: 1.8640 - val_acc: 0.3594 - val_f1_m: 0.3152
Epoch 47/500
221/221 [==============================] - 3s 12ms/step - loss: 0.9911 - acc: 0.6148 - f1_m: 0.5718 - val_loss: 1.8597 - val_acc: 0.3699 - val_f1_m: 0.3168
Epoch 48/500
221/221 [==============================] - 3s 12ms/step - loss: 0.9837 - acc: 0.6184 - f1_m: 0.5798 - val_loss: 1.9454 - val_acc: 0.3731 - val_f1_m: 0.3193
Epoch 49/500
221/221 [==============================] - 3s 12ms/step - loss: 0.9717 - acc: 0.6236 - f1_m: 0.5851 - val_loss: 1.8784 - val_acc: 0.3818 - val_f1_m: 0.3351
Epoch 50/500
221/221 [==============================] - 2s 11ms/step - loss: 0.9579 - acc: 0.6286 - f1_m: 0.5914 - val_loss: 1.9378 - val_acc: 0.3563 - val_f1_m: 0.3132
Epoch 51/500
221/221 [==============================] - 2s 11ms/step - loss: 0.9583 - acc: 0.6293 - f1_m: 0.5913 - val_loss: 2.0078 - val_acc: 0.3685 - val_f1_m: 0.3243

221/221 [==============================] - 2s 9ms/step - loss: 0.6301 - acc: 0.7629 - f1_m: 0.7527 - val_loss: 2.5503 - val_acc: 0.3723 - val_f1_m: 0.3650
Epoch 95/500
221/221 [==============================] - 2s 8ms/step - loss: 0.6198 - acc: 0.7667 - f1_m: 0.7553 - val_loss: 2.6049 - val_acc: 0.3577 - val_f1_m: 0.3349
Epoch 96/500
221/221 [==============================] - 2s 8ms/step - loss: 0.6204 - acc: 0.7663 - f1_m: 0.7546 - val_loss: 2.8657 - val_acc: 0.3236 - val_f1_m: 0.3155
Epoch 97/500
221/221 [==============================] - 2s 8ms/step - loss: 0.6152 - acc: 0.7688 - f1_m: 0.7569 - val_loss: 2.6687 - val_acc: 0.3568 - val_f1_m: 0.3434
Epoch 98/500
221/221 [==============================] - 1s 7ms/step - loss: 0.6040 - acc: 0.7712 - f1_m: 0.7626 - val_loss: 2.7052 - val_acc: 0.3626 - val_f1_m: 0.3464
Epoch 99/500
221/221 [==============================] - 1s 7ms/step - loss: 0.6072 - acc: 0.7705 - f1_m: 0.7606 - val_loss: 2.7514 - val_acc: 0.3522 - val_f1_m: 0.3365
Epoch

...

221/221 [==============================] - 2s 7ms/step - loss: 3.8481e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 0.0022 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 319/500
221/221 [==============================] - 1s 7ms/step - loss: 3.8128e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 0.0014 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 320/500
221/221 [==============================] - 1s 7ms/step - loss: 3.8158e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 0.0014 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 321/500
221/221 [==============================] - 1s 7ms/step - loss: 3.8086e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 0.0015 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 322/500
221/221 [==============================] - 1s 7ms/step - loss: 3.7893e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 0.0017 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 323/500
221/221 [==============================] - 1s 6ms/step - loss: 3.7953e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 0.0018 - val_acc: 1.

221/221 [==============================] - 1s 3ms/step - loss: 3.5584e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 0.0019 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 366/500
221/221 [==============================] - 1s 3ms/step - loss: 3.5740e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 0.0019 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 367/500
221/221 [==============================] - 1s 3ms/step - loss: 3.5732e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 0.0013 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 368/500
221/221 [==============================] - 1s 3ms/step - loss: 3.5489e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 0.0017 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 369/500
221/221 [==============================] - 1s 3ms/step - loss: 3.5585e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 0.0016 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 370/500
221/221 [==============================] - 1s 3ms/step - loss: 3.5505e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 0.0014 - val_acc: 1.

221/221 [==============================] - 1s 3ms/step - loss: 3.3649e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 0.0016 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 413/500
221/221 [==============================] - 1s 3ms/step - loss: 3.3734e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 0.0018 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 414/500
221/221 [==============================] - 1s 3ms/step - loss: 3.3735e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 0.0014 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 415/500
221/221 [==============================] - 1s 3ms/step - loss: 3.3589e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 0.0015 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 416/500
221/221 [==============================] - 1s 3ms/step - loss: 3.3447e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 0.0011 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 417/500
221/221 [==============================] - 1s 3ms/step - loss: 3.3520e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 0.0015 - val_acc: 1.

221/221 [==============================] - 2s 10ms/step - loss: 3.1985e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 0.0018 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 460/500
221/221 [==============================] - 2s 9ms/step - loss: 3.1950e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 0.0018 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 461/500
221/221 [==============================] - 2s 7ms/step - loss: 3.2449e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 0.0012 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 462/500
221/221 [==============================] - 1s 6ms/step - loss: 3.1659e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 0.0012 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 463/500
221/221 [==============================] - 1s 7ms/step - loss: 3.1813e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 0.0014 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 464/500
221/221 [==============================] - 1s 6ms/step - loss: 3.2052e-04 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 0.0014 - val_acc: 1

Fold #5
Accuracy = 0.9091605001275835, F1-score = 0.909256752189291, MCC = 0.9061934948075766, Train Time = 8.957908391952515
Train shape (70539, 117)
Test shape (7838, 117)
Fold #6
Accuracy = 0.9131155907119163, F1-score = 0.913059324299456, MCC = 0.910243616732593, Train Time = 8.870972156524658
Train shape (70539, 117)
Test shape (7838, 117)
Fold #7
Accuracy = 0.9077570808879817, F1-score = 0.9076923522016497, MCC = 0.904737262262796, Train Time = 8.986639022827148
Train shape (70540, 117)
Test shape (7837, 117)
Fold #8
Accuracy = 0.9094041087150695, F1-score = 0.9093309251547167, MCC = 0.9064237319711329, Train Time = 8.844229936599731
Train shape (70540, 117)
Test shape (7837, 117)
Fold #9
Accuracy = 0.9115733061120327, F1-score = 0.9115407338813671, MCC = 0.9086736867801349, Train Time = 8.816070556640625
Train shape (70540, 117)
Test shape (7837, 117)
Fold #10
Accuracy = 0.9143805027433967, F1-score = 0.9143138055347054, MCC = 0.9115457639751451, Train Time = 8.809188604354858
P